In [1]:
from tqdm import tqdm

import torch
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score

import torch_geometric.transforms as T
from torch_geometric.datasets import SNAPDataset
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

C:\Users\Abhijit\Documents\GitHub\cpsc490\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    T.ToDevice(device),
    T.RandomLinkSplit(
        num_val=0.05, 
        num_test=0.1, 
        is_undirected=True, 
        add_negative_train_samples=False
    ),
])

dataset = SNAPDataset(root="../data/SNAPDataset", name="ego-facebook", transform=transform)

train_data, val_data, test_data = dataset[0]

In [3]:
print(train_data)
print(val_data)
print(test_data)

Data(x=[347, 1406], edge_index=[2, 4864], circle=[325], circle_batch=[325], edge_label=[2432], edge_label_index=[2, 2432])
Data(x=[347, 1406], edge_index=[2, 4864], circle=[325], circle_batch=[325], edge_label=[284], edge_label_index=[2, 284])
Data(x=[347, 1406], edge_index=[2, 5148], circle=[325], circle_batch=[325], edge_label=[570], edge_label_index=[2, 570])


In [4]:
from torch import nn
import torch.nn.functional as F


class SimpleNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
        self.W1 = nn.Linear(out_channels * 2, out_channels)
        self.W2 = nn.Linear(out_channels, 1)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        z = torch.cat((z[edge_label_index[0]], z[edge_label_index[1]]), dim=1)
        return self.W2(F.relu(self.W1(z)).squeeze())

simple_model = SimpleNet(dataset.num_features, 128, 32).to(device)
simple_optimizer = torch.optim.Adam(params=simple_model.parameters(), lr=3e-3)
    
model = Net(dataset.num_features, 128, 32).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-3)
criterion = torch.nn.BCEWithLogitsLoss()

In [5]:
def train(model, optimizer, data):
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=data.edge_index, 
        num_nodes=data.num_nodes,
        num_neg_samples=data.edge_label_index.shape[1], 
        method='sparse'
    )

    edge_label_index = torch.cat([data.edge_label_index, neg_edge_index], dim=-1)
    edge_label = torch.cat([data.edge_label, data.edge_label.new_zeros(neg_edge_index.size(1))], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(model, data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    a, b = data.edge_label.cpu().numpy(), out.cpu().numpy()
    c = (out > 0.5).float().cpu().numpy()
    return roc_auc_score(a, b), accuracy_score(a, c)

In [6]:
best_val_auc = final_test_auc = final_test_acc = 0
for epoch in range(1, 1001):
    loss = train(simple_model, simple_optimizer, train_data)
    val_auc, val_acc = test(simple_model, val_data)
    test_auc, test_acc = test(simple_model, test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
        final_test_acc = test_acc
    if epoch % 50 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f} {val_acc:.4f}, Test: {test_auc:.4f} {test_acc:.4f}')

print(f'Final Test: {final_test_auc:.4f} {final_test_acc:.4f}')

simple_z = simple_model.encode(test_data.x, test_data.edge_index)
simple_final_edge_index = simple_model.decode(simple_z, test_data.edge_label_index)

Epoch: 050, Loss: 0.4899, Val: 0.9216 0.7394, Test: 0.9143 0.7456
Epoch: 100, Loss: 0.4625, Val: 0.9238 0.7394, Test: 0.9133 0.7561
Epoch: 150, Loss: 0.4511, Val: 0.9220 0.7289, Test: 0.9145 0.7702
Epoch: 200, Loss: 0.4513, Val: 0.9192 0.7359, Test: 0.9071 0.7772
Epoch: 250, Loss: 0.4371, Val: 0.9158 0.7535, Test: 0.8995 0.7842
Epoch: 300, Loss: 0.4279, Val: 0.9095 0.7500, Test: 0.8962 0.7789
Epoch: 350, Loss: 0.4174, Val: 0.9185 0.7430, Test: 0.9020 0.7684
Epoch: 400, Loss: 0.4305, Val: 0.9041 0.7430, Test: 0.8963 0.7947
Epoch: 450, Loss: 0.4292, Val: 0.9031 0.7430, Test: 0.8966 0.7860
Epoch: 500, Loss: 0.4210, Val: 0.9103 0.7570, Test: 0.8959 0.7982
Epoch: 550, Loss: 0.4195, Val: 0.9009 0.7500, Test: 0.8938 0.7912
Epoch: 600, Loss: 0.4087, Val: 0.9067 0.7641, Test: 0.8961 0.7877
Epoch: 650, Loss: 0.4149, Val: 0.9015 0.7606, Test: 0.8942 0.7842
Epoch: 700, Loss: 0.3894, Val: 0.8985 0.7782, Test: 0.8894 0.7807
Epoch: 750, Loss: 0.4012, Val: 0.9054 0.7817, Test: 0.8920 0.7842
Epoch: 800

In [7]:
best_val_auc = final_test_auc = final_test_acc = 0
for epoch in range(1, 1001):
    loss = train(model, optimizer, train_data)
    val_auc, val_acc = test(model, val_data)
    test_auc, test_acc = test(model, test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
        final_test_acc = test_acc
    if epoch % 50 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f} {val_acc:.4f}, Test: {test_auc:.4f} {test_acc:.4f}')

print(f'Final Test: {final_test_auc:.4f} {final_test_acc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode(z, test_data.edge_label_index)

Epoch: 050, Loss: 0.3604, Val: 0.9260 0.8486, Test: 0.9086 0.8211
Epoch: 100, Loss: 0.2921, Val: 0.9522 0.8732, Test: 0.9367 0.8544
Epoch: 150, Loss: 0.2633, Val: 0.9570 0.8803, Test: 0.9404 0.8632
Epoch: 200, Loss: 0.2588, Val: 0.9576 0.8908, Test: 0.9417 0.8754
Epoch: 250, Loss: 0.2462, Val: 0.9619 0.8979, Test: 0.9447 0.8754
Epoch: 300, Loss: 0.2099, Val: 0.9665 0.9049, Test: 0.9554 0.8947
Epoch: 350, Loss: 0.2147, Val: 0.9667 0.9014, Test: 0.9559 0.8912
Epoch: 400, Loss: 0.2008, Val: 0.9671 0.8979, Test: 0.9590 0.9000
Epoch: 450, Loss: 0.1932, Val: 0.9677 0.8873, Test: 0.9604 0.8930
Epoch: 500, Loss: 0.1881, Val: 0.9658 0.8979, Test: 0.9581 0.9000
Epoch: 550, Loss: 0.1846, Val: 0.9660 0.8979, Test: 0.9600 0.9000
Epoch: 600, Loss: 0.2057, Val: 0.9649 0.8908, Test: 0.9594 0.8965
Epoch: 650, Loss: 0.1881, Val: 0.9651 0.9014, Test: 0.9604 0.8947
Epoch: 700, Loss: 0.1959, Val: 0.9649 0.8944, Test: 0.9607 0.8947
Epoch: 750, Loss: 0.1780, Val: 0.9634 0.8944, Test: 0.9578 0.8982
Epoch: 800

In [8]:
(test_data.edge_label == (torch.sigmoid(simple_final_edge_index.squeeze()) > 0.7)).sum() / test_data.edge_label.shape[0]

tensor(0.8474)

In [9]:
(test_data.edge_label == (torch.sigmoid(final_edge_index.squeeze()) > 0.5)).sum() / test_data.edge_label.shape[0]

tensor(0.9000)